In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data assimilation - Understanding model and observation errors


In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

def plotGauss3D(mean_0,mean_1,sd_0,sd_1,rho,vminmax=None,title=None):
# size of grid
    N = 1000
#  generate grid (NB -1 to 1)
    coords = 2. * (np.arange(N+1)/float(N) - 0.5)
    x0, x1 = np.meshgrid(coords,coords)
    x = np.array([x0, x1])
    dx = np.array([x0[0,1] - x0[0,0], x1[1,0] - x1[0,0]])
    grid = dx[0] * dx[1]

    # set B
    b01 = b10 = rho * sd_0 * sd_1
    b00 = sd_0**2
    b11 = sd_1**2
    B = np.matrix([[b00,b01],[b10,b11]])
    # set xb: the mean
    xb = np.array([mean_0,mean_1])

    xxb = np.zeros_like(x)
    for i in range(xb.shape[0]): xxb[i,...] = xb[i]
    e = x - xxb

    n = np.shape(B)[0]
    # inverse of B
    BI = B.I
    # scaling term
    scale_1 = (2.*np.pi)**(n/2) * np.sqrt(np.linalg.det(B))
    gauss = np.exp(-0.5 * ((e[0,...] * BI[0,0] + e[1,...] * BI[0,1])* e[0,...]     \
                          + (e[0,...] * BI[1,0] + e[1,...] * BI[1,1])* e[1,...])) \
                          / scale_1

    # check integral
    #print ('integral of Gaussian:',gauss.sum() * grid)

    
    fig = plt.figure()
    ax = fig.gca(projection='3d')

    # Plot the surface.
    surf = ax.plot_surface(x0, x1, gauss, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

    # Customize the z axis.
    ax.set_zlim(-.0, 5.0)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

    # Add a color bar which maps values to colors.
    fig.colorbar(surf, shrink=0.5, aspect=5)
    if title:
        plt.title(title)
    plt.show()   
        
  

## Plot the Multivariate Gaussian Distribution

$P_b(x) = \frac{1}{(2 \pi)^\frac{n}{2} \sqrt{\det(B)}} \exp \left( - \frac{1}{2} (x_b - x)^T B^{-1} (x_b - x) \right)$

A dimensional example with 
$x_b = (0.2,0.5)$; $\sigma_b = (0.3,0.2)$; $\rho=-0.5$

In [ ]:
xb = [0.2,0.5]
sb = [0.3,0.2]
rho = -0.5
plotGauss3D(xb[0],xb[1],sb[0],sb[1],rho)

## Example for a 2-dimensional data assimilation problem:

In [ ]:
import numpy as np
import scipy.optimize

### (a)  Prior estimate (Model Background) $x_b$ and the B-Matrix

In [ ]:
xb = np.array([0.1,0.5]) # Model Background
sd_0 = 0.2   # Standard deviation of value 1
sd_1 = 0.3   # Standard deviation of value 2
pcorr = 0.5  # Pearson correlation coefficient
# The resulting B-Matrix
B  = np.matrix([[sd_0**2,pcorr*sd_0*sd_1],[pcorr*sd_0*sd_1,sd_1**2]])

### (b) Two observation and their uncertainties

In [ ]:
xr = np.array([0.15,0.4]) # Observation
sd = 0.2 # Standard deviation of the observation
# The resulting R-Matrix
R  = np.matrix([[sd**2,0.0],[0.0,sd**2]])

### (c) Define the functions
#### Cost ()  -> returns the cost function J and the Jacobian J' 
#### Uncertaintity()-> returns the Hessian J''

In [ ]:
def cost(x,xb,B,xr,R):
    Jb = np.dot(np.array(0.5*(xb-x) * B.I),(xb-x))[0]
    Jr = np.dot(np.array(0.5*(xr-x) * R.I),(xr-x))[0]
    JbPrime = -(xb-x)*B.I
    JrPrime = -(xr-x)*R.I
    return Jr+Jb,np.array(JrPrime+JbPrime)[0]

def uncertainty(x,xb,B,xr,R):
    return (B.I + R.I).I

### (d) Algorithm to find the optimal solution x

In [ ]:
# starting guess
x = np.array([0.,0.])
retval = scipy.optimize.fmin_l_bfgs_b(cost,x,args=(xb,B,xr,R))
# x new
x = retval[0]
# uncertainty
Cpost = uncertainty(x,xb,B,xr,R)

print ('Results:')
# print prior
psigma0 = np.sqrt(B[0,0]); psigma1 = np.sqrt(B[1,1])
prho12  = B[0,1]/(psigma0*psigma1)
print ('Model background:     xb      =',xb[0],xb[1])
print ('                      sigma   =',psigma0,psigma1)
print ('                      rho     =',prho12)

# print observation
rsigma0 = np.sqrt(R[0,0]); rsigma1 = np.sqrt(R[1,1])
rrho12  = R[0,1]/(rsigma0*rsigma1)
print ('Observation:          xo      =',xr[0],xr[1])
print ('                      sigma   =',rsigma0,rsigma1) 
print ('                      rho     =',rrho12)

sigma0 = np.sqrt(Cpost[0,0]); sigma1 = np.sqrt(Cpost[1,1])
rho12  = Cpost[0,1]/(sigma0*sigma1)
print ('Solution:             x       =',x[0],x[1])
print ('                      sigma   =',sigma0,sigma1)
print ('                      rho     =',rho12)

### (e) Plot the Gaussian distriubtions before and after data assimilation 

In [ ]:
plotGauss3D(xb[0],xb[1],psigma0,psigma1,prho12,vminmax=[0,5],title='prior')

In [ ]:
plotGauss3D(xr[0],xr[1],rsigma0,rsigma1,rrho12,vminmax=[0,3],title='observation')

In [ ]:
plotGauss3D(x[0],x[1],sigma0,sigma1,rho12,vminmax=[0,5],title='posterior')